Truy cập vào Google Drive để lấy thư mục chứa dữ liệu

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
path = '/content/drive/MyDrive/Colab Notebooks/Fruits360'
print(os.listdir(path))

['Lite 14.03.2025', 'Lite 15.03.2025']


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Trỏ đường dẫn đến thư mục chứa dữ liệu

In [ ]:
train_dir = path + '/Training'
validation_dir = path + '/Test'

Tạo generator cho dữ liệu training và validation

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical'
)

Found 9909 images belonging to 19 classes.
Found 3309 images belonging to 19 classes.


Xây dựng mô hình CNN

In [ ]:
model_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(19, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Biên dịch mô hình

In [ ]:
model_cnn.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Tối ưu hoá mô hình

Huấn luyện mô hình

In [ ]:
model_cnn.fit(train_generator, epochs=10, validation_data=validation_generator)

Epoch 1/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.6506 - loss: 1.2791

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


310/310 ━━━━━━━━━━━━━━━━━━━━ 3247s 10s/step - accuracy: 0.6513 - loss: 1.2764 - val_accuracy: 0.9749 - val_loss: 0.0556
Epoch 2/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 208s 671ms/step - accuracy: 0.9937 - loss: 0.0195 - val_accuracy: 0.9341 - val_loss: 0.2393
Epoch 3/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 189s 609ms/step - accuracy: 0.9910 - loss: 0.0307 - val_accuracy: 0.9858 - val_loss: 0.0381
Epoch 4/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 188s 607ms/step - accuracy: 0.9983 - loss: 0.0042 - val_accuracy: 0.9680 - val_loss: 0.0833
Epoch 5/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 187s 604ms/step - accuracy: 0.9983 - loss: 0.0043 - val_accuracy: 0.9761 - val_loss: 0.0563
Epoch 6/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 189s 611ms/step - accuracy: 0.9999 - loss: 4.3173e-04 - val_accuracy: 0.9716 - val_loss: 0.0830
Epoch 7/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 192s 618ms/step - accuracy: 1.0000 - loss: 1.3315e-04 - val_accuracy: 0.9770 - val_loss: 0.0477
Epoch 8/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 200s 612ms/step - accuracy: 1.0000 - los

Chuyển đổi mô hình sang Tensorflow Lite

In [ ]:
import tensorflow as tf
converter_tflite_model = tf.lite.TFLiteConverter.from_keras_model(model_cnn)
tflite_model = converter_tflite_model.convert()

Saved artifact at '/tmp/tmpsiq8uuix'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 100, 3), dtype=tf.float32, name='keras_tensor_16')
Output Type:
  TensorSpec(shape=(None, 19), dtype=tf.float32, name=None)
Captures:
  139864518832784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139864518834512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139864518833936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139864518835280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139864518834704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139864518835856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139864518836048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139864518833744: TensorSpec(shape=(), dtype=tf.resource, name=None)


Tải mô hình

In [ ]:
os.makedirs('/content/drive/MyDrive/Colab Notebooks/Results', exist_ok=True)

with open('/content/drive/MyDrive/Colab Notebooks/Results/fruits360_model.tflite', 'wb') as f:
    f.write(tflite_model)

Hiển thị số lớp trong mô hình

In [ ]:
print(train_generator.class_indices)

{'Apple 10': 0, 'Apple 12': 1, 'Apple 13': 2, 'Apple 14': 3, 'Apple 6': 4, 'Apple Braeburn 1': 5, 'Apple Crimson Snow 1': 6, 'Apple Golden 1': 7, 'Apple Golden 2': 8, 'Apple Golden 3': 9, 'Apple Granny Smith 1': 10, 'Apple Pink Lady 1': 11, 'Apple Red 1': 12, 'Apple Red 2': 13, 'Apple Red 3': 14, 'Apple Red Delicious 1': 15, 'Apple Red Yellow 1': 16, 'Apple Red Yellow 2': 17, 'Apple hit 1': 18}


Xuất file .h5

In [ ]:
model_cnn.save('/content/drive/MyDrive/Colab Notebooks/Results/fruits360_model.keras')

Thử nghiệm mô hình với h5

In [ ]:
import cv2 as cv
import numpy as np

class_names = [
    'Apple 10', 'Apple 12', 'Apple 13', 'Apple 14', 'Apple 6',
    'Apple Braeburn 1', 'Apple Crimson Snow 1', 'Apple Golden 1',
    'Apple Golden 2', 'Apple Golden 3', 'Apple Granny Smith 1',
    'Apple Pink Lady 1', 'Apple Red 1', 'Apple Red 2', 'Apple Red 3',
    'Apple Red Delicious 1', 'Apple Red Yellow 1', 'Apple Red Yellow 2',
    'Apple hit 1'
]

model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Results/fruits360_model.h5')
image = cv.imread('/content/drive/MyDrive/Colab Notebooks/r1_295_100.jpg')
image = cv.resize(image, (100, 100)) / 255.0
image = np.expand_dims(image, axis=0).astype(np.float32)
predictions = model.predict(image)
print(predictions[0])

class_idx = np.argmax(predictions[0])
class_name = class_names[class_idx]
confidence = predictions[0][class_idx] * 100
print(f"Result: {class_name}: {confidence:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
[1.06870407e-32 9.99995828e-01 4.84821889e-18 4.49050966e-26
 4.17088313e-06 5.95673476e-23 6.41732657e-21 1.23328275e-20
 7.52201925e-20 2.51357123e-27 1.93296707e-17 1.47186513e-32
 7.34631820e-19 1.84433287e-37 7.37712474e-34 1.93531629e-14
 0.00000000e+00 6.81574859e-13 3.05131072e-12]
Result: Apple 12: 100.00%
